In [11]:
contents = ["大巨蛋案對市府同仁下封口令？柯P否認", "柯P市府近來飽受大巨蛋爭議"]

In [19]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
for sentence in contents:
    #print(list(jieba.cut(sentence)))
    corpus.append(' '.join(jieba.cut(sentence)))

In [20]:
corpus

['大巨蛋 案對 市府 同仁 下 封口令 ？ 柯P 否認', '柯P 市府 近來 飽受 大巨蛋 爭議']

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<2x10 sparse matrix of type '<class 'numpy.longlong'>'
	with 13 stored elements in Compressed Sparse Row format>

In [25]:
print(vectorizer.get_feature_names())

['同仁', '否認', '大巨蛋', '封口令', '市府', '柯p', '案對', '爭議', '近來', '飽受']


In [24]:
X.toarray()

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 1, 1, 1]], dtype=int64)

In [26]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
cosine_similarity(X)

array([[1.        , 0.46291005],
       [0.46291005, 1.        ]])

In [27]:
cosine_distances(X)

array([[0.        , 0.53708995],
       [0.53708995, 0.        ]])

## 根據新聞相似度推薦新聞

In [30]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/20171214news.xlsx', index_col = 0)

In [36]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
titles = []
for idx, rec in news.iterrows():
    corpus.append(' '.join(jieba.cut(rec.get('content'))))
    titles.append(rec.get('title'))

In [38]:
#corpus[0]
titles[0]

'【更新】水利會改官派明闖關\u3000綠委24小時前顧議場大門防藍突襲'

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [41]:
X

<899x28957 sparse matrix of type '<class 'numpy.longlong'>'
	with 122298 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)
cs.shape

(899, 899)

In [46]:
titles[12]

'黃國昌：長期目標要消滅國民黨\u3000不支持柯文哲的兩岸一家親'

In [48]:
cs[12].shape

(899,)

In [52]:
import numpy 
students = [60,30,40,70,90,80,]
students_ary = numpy.array(students)
students_ary.argsort()

array([1, 2, 0, 3, 5, 4])

In [56]:
for idx in cs[12].argsort()[0:10]:
    print(titles[idx], cs[12][idx])

黃國昌：長期目標要消滅國民黨　不支持柯文哲的兩岸一家親 0.0
黃國昌若被罷免會更強　他：成為選輸北市的阿扁 0.5542565596535691
罷免案將投票　李遠哲今再度現身力挺黃國昌 0.5752596978859429
「兩蔣時代」超譯　網友：不准傷皇城內的和氣！ 0.5938835126582123
罷昌案周六投票　時代力量全力澄清不實謠言 0.6402804718905146
【聲援片】沈發惠呼籲罷昌案投「不同意」　黃國昌：謝謝您 0.7003246342755608
宗教團體可曾想過：黃國昌犀利形象恐越打越旺 0.7749027801375493
【歷史上的今天】幫黃國昌站台　蔡英文：我有準備當總統了 0.7803630780883579
【更新】水利會改官派明闖關　綠委24小時前顧議場大門防藍突襲 0.7869878365124581
臉書在台灣繳稅太少　財政部將進一步追稅 0.7926505336912069


In [58]:
pos = 12
print('查詢文章:',titles[pos])
for idx in cs[pos].argsort()[1:10]:
    if cs[pos][idx] < 0.7:
        print('相關文章',titles[idx], cs[pos][idx])

查詢文章: 黃國昌：長期目標要消滅國民黨　不支持柯文哲的兩岸一家親
相關文章 黃國昌若被罷免會更強　他：成為選輸北市的阿扁 0.5542565596535691
相關文章 罷免案將投票　李遠哲今再度現身力挺黃國昌 0.5752596978859429
相關文章 「兩蔣時代」超譯　網友：不准傷皇城內的和氣！ 0.5938835126582123
相關文章 罷昌案周六投票　時代力量全力澄清不實謠言 0.6402804718905146


In [59]:
def getSimiliarNews(pos):
    print('查詢文章:',titles[pos])
    for idx in cs[pos].argsort()[1:10]:
        if cs[pos][idx] < 0.7:
            print('相關文章',titles[idx], cs[pos][idx])

In [65]:
getSimiliarNews(35)

查詢文章: FBI探員罵川普「白痴」　遭調離通俄門調查
相關文章 【公道難伸張】再控川普性騷　受害者：沒人在乎很受傷 0.6434278035355978
相關文章 老謝：美菲2位狂人兄弟 今年交出不錯成績單　 0.6502913209540035
相關文章 否認性騷指控　川普：民主黨編的假新聞 0.6754198672772045
相關文章 諷參議員陸天娜「為錢啥都做」　川普遭美媒批不適任總統 0.6984356814409867


In [67]:
q = '黃國昌 李遠哲'
q_list = [' '.join(jieba.cut(q))]

In [71]:
q_vec = vectorizer.transform(q_list)
q_vec

<1x28957 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [73]:
q_cs = cosine_distances(q_vec, X)

In [74]:
q_cs.shape

(1, 899)

In [79]:
q_cs.argsort()[0][0]

182

In [80]:
titles[182]

'黃國昌若被罷免會更強\u3000他：成為選輸北市的阿扁'

## 小任務(一)
- 請先寫個網路爬蟲爬取台北富邦銀行的常見問題，並將問題與答案整理名為qa的DataFrame？
- https://www.fubon.com/banking/faq/index.htm

## 小任務
- 試著建立一個依內容相似度回應答案的問答機器人，當使用者輸入問題時，機器人可以依相似問題給予對應答案
